# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [2]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [1]:
# ваш код
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
import math

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__
    __radd__ = __add__

    # унарные
    def __pos__(self) -> "Dual":
        return self
    def __neg__(self) -> "Dual":
        return Dual(-1 * self.value, -1 * self.d)
        
    # деление
    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value / self.value, (self.value * o_d - self.d * o_value) / (self.value * self.value))
            case Number():
                return Dual(float(other) / self.value,  -1 * float(other) / (self.value * self.value) * self.d)

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / (o_value * o_value))
            case Number():
                return Dual(self.value / float(other),  self.d / float(other))

    # возведение в степень
    def __pow__(self, other: Union["Dual", Number], modulo=None):
        match other:
            case Dual(o_value, o_d):
                raise NotImplementedError("**")
            case Number():
                return Dual(self.value ** float(other), float(other) * self.value ** (float(other) - 1))
    def __rpow__(self, other: Union["Dual", Number], modulo=None):
        match other:
            case Dual(o_value, o_d):
                raise NotImplementedError("**")
            case Number():
                return Dual(float(other) ** self.value, math.log(float(other), math.e) *  float(other) ** self.value)
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

In [27]:
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2
f_diff = diff(f)(2)
assert f_diff == 22
print(f'Полученное значение: {f_diff}, реальное значение: 22')

def f(x: float) -> float:
    return -x
f_diff = diff(f)(2)
assert f_diff == -1
print(f'Полученное значение: {f_diff}, реальное значение: -1')

def f(x: float) -> float:
    return 4 / x + x
f_diff = diff(f)(2)
assert f_diff == 0
print(f'Полученное значение: {f_diff}, реальное значение: 0')

def f(x: float) -> float:
    return x**2 / 4
f_diff = diff(f)(4)
assert f_diff == 2
print(f'Полученное значение: {f_diff}, реальное значение: 2')

def f(x: float) -> float:
    return 2**x
f_diff = diff(f)(2)
assert f_diff - 2.77 < 10e-2
print(f'Полученное значение: {f_diff}, реальное значение: 2.77')

Полученное значение: 22.0, реальное значение: 22
Полученное значение: -1.0, реальное значение: -1
Полученное значение: 0.0, реальное значение: 0
Полученное значение: 2.0, реальное значение: 2
Полученное значение: 2.772588722239781, реальное значение: 2.77


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [7]:

def exp(x: Union["Dual", Number]) -> Union["Dual", Number]:
    match x:
        case Dual(value, d):
            return Dual(math.exp(value), d * math.exp(value))
        case Number():
            return math.exp(x)

def cos(x: Union["Dual", Number]) -> Union["Dual", Number]:
    match x:
        case Dual(value, d):
            return Dual(math.cos(value), -math.sin(value) * d)
        case Number():
            return math.cos(x)

def sin(x: Union["Dual", Number]) -> Union["Dual", Number]:
    match x:
        case Dual(value, d):
            return Dual(math.sin(value), math.cos(value) * d)
        case Number():
            return math.sin(x)

def log(x: Union["Dual", Number]) -> Union["Dual", Number]:
    match x:
        case Dual(value, d):
            return Dual(math.log(value), d / value)
        case Number():
            return math.log(x)

In [43]:
def f(x: float) -> float:
    return sin(x)
f_diff = diff(f)(0)
assert f_diff == 1
print(f'Полученное значение: {f_diff}, реальное значение: 1')

def f(x: float) -> float:
    return cos(x)
f_diff = diff(f)(0)
assert f_diff == 0
print(f'Полученное значение: {f_diff}, реальное значение: 0')

def f(x: float) -> float:
    return exp(x**2)
f_diff = diff(f)(2)
assert f_diff - 218.39 < 10e-2
print(f'Полученное значение: {f_diff}, реальное значение: 218.39')

def f(x: float) -> float:
    return log(x**3)
f_diff = diff(f)(2)
assert f_diff == 1.5
print(f'Полученное значение: {f_diff}, реальное значение: 1.5')

Полученное значение: 1.0, реальное значение: 1
Полученное значение: -0.0, реальное значение: 0
Полученное значение: 218.39260013257694, реальное значение: 218.39
Полученное значение: 1.5, реальное значение: 1.5


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [5]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [44]:
# ваш код
def num_diff(f):
    def diff_f(x):
        dx = 0.001
        df = f(x + dx) - f(x)
        df_dx = df / dx
        while True:
            dx = 0.5 * dx
            df = f(x + dx) - f(x)
            df_dx_new = df / dx
            if dx < 0.000001:
                return df_dx_new
            if abs(df_dx_new - df_dx) < 10e-6:
                return df_dx_new
            df_dx = df_dx_new
    return diff_f

In [45]:
def f(x: float) -> float:
    return sin(x)
f_diff = num_diff(f)(0)
assert f_diff - 1 < 10e-3

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [86]:
# ваш код
import random

def random_f():
    stroka = 'lambda x: '

    dict_needed_str = {
        'brackets' : '()', 
        'used_signs' : [' + ', ' * ', ' / '], 
        'fun' : ['sin', 'cos', 'log', 'exp']
        }

    for i in range(10):
        random_n = random.randint(0, len(dict_needed_str)-1)
        if random_n == 0:
            stroka += 'x' + random.choice(dict_needed_str['used_signs'])
        elif random_n == 1:
            stroka += str(random.uniform(-10, 10)) + random.choice(dict_needed_str['used_signs'])
        else:
            stroka += random.choice(dict_needed_str['fun']) + dict_needed_str['brackets'][0] + 'x' + dict_needed_str['brackets'][1]  + random.choice(dict_needed_str['used_signs'])

    stroka += 'x'
    return stroka

In [87]:
for i in range(100):
    print('№ ', i)
    str_fun = random_f()
    print('Generated Function: ', str_fun)
    sym_diff = diff(eval(str_fun))(2)
    n_diff = num_diff(eval(str_fun))(2)
    print(f'sym_diff: {sym_diff}, n_diff: {n_diff} \n')
    assert abs(sym_diff - n_diff) < 10e-2

№  0
Generated Function:  lambda x: exp(x) / -0.09457150911650913 / x + x + x + sin(x) + cos(x) + -6.447652645481541 + x + -3.0759050636976966 / x
sym_diff: -17.089454644165556, n_diff: -17.08946479629958 

№  1
Generated Function:  lambda x: x + 8.537998536077719 + x / x / exp(x) + 1.759936672102377 / 8.045500395494258 / x + exp(x) + 5.7606732522750566 * x
sym_diff: 13.959707081902696, n_diff: 13.959714486190933 

№  2
Generated Function:  lambda x: x * x * sin(x) / log(x) * x / x / x * sin(x) + x * 0.48308714287091803 * x
sym_diff: -0.7793913857854693, n_diff: -0.7793981613133383 

№  3
Generated Function:  lambda x: log(x) + 8.072005427530527 * cos(x) / -7.131226550399832 / x / cos(x) + sin(x) * -8.35351427079474 / log(x) + x * x
sym_diff: 17.703066662757475, n_diff: 17.703056734262645 

№  4
Generated Function:  lambda x: 8.136367961615957 * x * x + 2.9946177035514854 / x * log(x) + x * x + sin(x) / x * x
sym_diff: 36.359051731286655, n_diff: 36.35905991541222 

№  5
Generated Func

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [58]:
# ваш код

def diff_multy(func: Callable):
    def r(*args):
        res = []
        for i in range(len(args)):
            diff_i = list(args)
            diff_i[i] = Dual(args[i], 1.0)
            diff_i = tuple(diff_i)
            p = func(*diff_i).d
            res.append(p)
        return res
    return r

In [61]:
def f_m(x,y,z):
    return x**2 + y**3 + z**4 + x + y + z
f_m_diff = diff_multy(f_m)
print(f_m_diff(1,1,1))

[3.0, 4.0, 5.0]
